In [1]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    load_index_from_storage
)
load_dotenv()
# openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="Qwen/Qwen3-Embedding-0.6B")
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [9]:
PERSIST_DIR = "local_index2"

if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

Parsing nodes:   0%|          | 0/104 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/115 [00:00<?, ?it/s]

In [10]:
query_engine = index.as_query_engine()

In [11]:
response = query_engine.query("Cuantos casos de embarazo adolescente se presentaron en 2022 en Bolivia?")


In [12]:
response.response


'En 2022, se presentaron un total de 35,250 casos de embarazo adolescente en Bolivia.'

In [13]:
embeddings = embed_model.get_text_embedding("hola bola")

In [14]:
print(len(embeddings))
print(embeddings[:5])

1024
[0.01754576340317726, -0.023225946351885796, -0.012997476384043694, -0.05905592069029808, 0.03175603598356247]


In [15]:
prompts_dict = query_engine.get_prompts()

In [17]:
prompts_dict['response_synthesizer:text_qa_template']

SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x160b2a3e0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert

In [18]:
from IPython.display import Markdown, display

In [19]:
for k, p in prompts_dict.items():
    text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
    display(Markdown(text_md))
    print(p.get_template())
    display(Markdown("<br><br>"))

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [30]:
new_qa_prompt = """La información del contexto es la siguiente.\n
---------------------\n
{context_str}\n
---------------------\n
Dado el contexto y sin ninguna información a priori, 
Responda la pregunta del usuario como si fuera una noticia de TV. Incluya detalles del contexto como ser número de página y documento fuente.\n
Query: {query_str}\n
Answer: """
from llama_index.core.prompts import PromptTemplate
qa_template = PromptTemplate(template=new_qa_prompt)

In [31]:
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_template})

In [32]:
response = query_engine.query("Cuantos casos de embarazo adolescente se presentaron en 2022 en Bolivia?")

In [33]:
display(Markdown(response.response))

Según el informe de avance de Bolivia del año 2022, en total se registraron 35,250 casos de embarazo adolescente. De estos, 2,119 correspondían al grupo de edad de 10 a 14 años, mientras que 33,131 eran embarazos del grupo de 15 a 19 años. A pesar de que estas cifras han disminuido desde el año 2015, la prevalencia de embarazos en adolescentes, especialmente en niñas, sigue siendo persistente. Esta información se puede encontrar en la página 33 del documento "Informe de Avance Bolivia".